In [2]:
import os
import gc
import json
import argparse
import numpy as np
import pandas as pd

from ray import tune
from evaluate import load
from torch.utils.data import DataLoader
from ray.tune.schedulers import PopulationBasedTraining
from transformers import TrOCRProcessor, VisionEncoderDecoderModel, default_data_collator, \
                         Seq2SeqTrainer, Seq2SeqTrainingArguments

from src.dataload import IAMDataset


def get_names_and_np(folder:str):
    data = []
    for fn in os.listdir(os.path.join(folder, 'img')):
        with open(os.path.join(folder, 'ann', f"{fn.rsplit('.', 1)[0]}.json"), 'r') as f:
            js_data = json.load(f)
            data.append([fn, js_data['description']])
    return data

def get_df_from_folder(train_folder:str, val_folder:str, test_folder:str, columns=['file_name', 'text']):
    train_data, val_data, test_data = get_names_and_np(train_folder), \
                                      get_names_and_np(val_folder),   \
                                      get_names_and_np(test_folder)
    train_df, val_df, test_df = pd.DataFrame(train_data, columns=columns), \
                                pd.DataFrame(val_data, columns=columns),   \
                                pd.DataFrame(test_data, columns=columns)
    return train_df, val_df, test_df


def update_model_config(model):
    # set special tokens used for creating the decoder_input_ids from the labels
    model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
    model.config.pad_token_id = processor.tokenizer.pad_token_id
    # make sure vocab size is set correctly
    model.config.vocab_size = model.config.decoder.vocab_size

    # set beam search parameters
    model.config.eos_token_id = processor.tokenizer.sep_token_id
    model.config.max_length = 64
    model.config.early_stopping = True
    model.config.no_repeat_ngram_size = 3
    model.config.length_penalty = 2.0
    model.config.num_beams = 4
    return model


def model_init(trial):
    model = VisionEncoderDecoderModel.from_pretrained(
        'microsoft/trocr-large-stage1')
    model = update_model_config(model)
    return model


def compute_metrics(pred):
    cer_metric = load('cer')
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(labels_ids, skip_special_tokens=True)
    cer_metric = load("cer")
    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return {"cer": cer}


if __name__ == '__main__':
    
    dict_ocr_folders = {'Ru': '/nfs/home/isaitov/NL/data/autoriaNumberplateOcrRu-2021-09-01/',
                        'Am': '/nfs/home/isaitov/NL/data/autoriaNumberplateOcrAm-2022-11-21-all/',
                        'Eu': '/nfs/home/isaitov/NL/data/autoriaNumberplateOcrEu-2023-04-25/',
                        'Kg': '/nfs/home/isaitov/NL/data/autoriaNumberplateOcrKg-2022-11-30/',
                        'Kz': '/nfs/home/isaitov/NL/data/autoriaNumberplateOcrKz-2022-11-29/',
                        'Su': '/nfs/home/isaitov/NL/data/autoriaNumberplateOcrSu-2023-03-10/',
                        'Ge': '/nfs/home/isaitov/NL/data/autoriaNumberplateOcrGe-2022-11-29/',
                        'Ua': '/nfs/home/isaitov/NL/data/autoriaNumberplateOcrUa-2023-04-18/',
                        'Md': '/nfs/home/isaitov/NL/data/autoriaNumberplateOcrMd-2023-01-27/',
                        'By': '/nfs/home/isaitov/NL/data/autoriaNumberplateOcrBy-2021-08-27/'}
    
    cer_metric = load('cer')
    processor = TrOCRProcessor.from_pretrained("microsoft/trocr-large-handwritten")
    
    for key in dict_ocr_folders:
    
        ocr_folder = dict_ocr_folders[key]
        train_ocr_folder, val_ocr_folder, test_ocr_folder = os.path.join(ocr_folder, 'train'),  \
                                                            os.path.join(ocr_folder, 'val'),   \
                                                            os.path.join(ocr_folder, 'test')

        try:
            train_df = pd.read_csv(os.path.join(ocr_folder, 'train_df.csv'))
            val_df = pd.read_csv(os.path.join(ocr_folder, 'val_df.csv'))
            test_df = pd.read_csv(os.path.join(ocr_folder, 'test_df.csv'))
        except:
            print('Error. re-init')
            train_df, val_df, test_df = get_df_from_folder(train_ocr_folder, val_ocr_folder, test_ocr_folder)
            train_df.to_csv(os.path.join(ocr_folder, 'train_df.csv'), index=False)
            val_df.to_csv(os.path.join(ocr_folder, 'val_df.csv'), index=False)
            test_df.to_csv(os.path.join(ocr_folder, 'test_df.csv'), index=False)
        
        train_dataset = IAMDataset(root_dir=os.path.join(train_ocr_folder, 'img/'),
                                   df=train_df,
                                   processor=processor)
        val_dataset = IAMDataset(root_dir=os.path.join(val_ocr_folder, 'img/'),
                                   df=val_df,
                                   processor=processor)
        test_dataset = IAMDataset(root_dir=os.path.join(test_ocr_folder, 'img/'),
                                   df=test_df,
                                   processor=processor)
        
        training_args = Seq2SeqTrainingArguments(
            output_dir=f'exps/aug__large_{key}',
            predict_with_generate=True,
            evaluation_strategy="steps",
            per_device_train_batch_size=2,
            per_device_eval_batch_size=8,
            load_best_model_at_end=True,
            fp16=True,
            logging_steps=2,
            save_steps=1000,
            eval_steps=200,
            max_steps=1500
        )

        # instantiate trainer
        trainer = Seq2SeqTrainer(
            model=None,
            tokenizer=processor.feature_extractor,
            model_init=model_init,
            args=training_args,
            compute_metrics=compute_metrics,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            data_collator=default_data_collator,
        )
        
        print("Running evaluation...")
        
        trainer.train()
        
        # test eval
        test_dataloader = DataLoader(test_dataset, batch_size=8)
        for batch in tqdm(test_dataloader):
            # predict using generate
            pixel_values = batch["pixel_values"].to('cuda' if torch.cuda.is_available() else 'cpu')
            outputs = model.generate(pixel_values)

            # decode
            pred_str = processor.batch_decode(outputs, skip_special_tokens=True)
            labels = batch["labels"]
            labels[labels == -100] = processor.tokenizer.pad_token_id
            label_str = processor.batch_decode(labels, skip_special_tokens=True)

            # add batch to metric
            cer_metric.add_batch(predictions=pred_str, references=label_str)

        final_score = cer_metric.compute()
        print(f"Character error rate on test {key} set:", final_score)
        
        # free memory
        try: 
            trainer.model.to('cpu')
        except:
            pass
        del trainer
        gc.collect()
        with torch.no_grad():
            torch.cuda.empty_cache()

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
/nfs/home/isaitov/miniconda3/envs/nl39/lib/python3.9/site-packages/transformers/models/trocr/processing_trocr.py:134: FutureWarning: `feature_extractor` is deprecated and will be removed in v5. Use `image_processor` instead.
  warnings.warn(
Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-large-stage1 and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Running evaluation...


Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-large-stage1 and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/nfs/home/isaitov/miniconda3/envs/nl39/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/nfs/home/isaitov/miniconda3/envs/nl39/lib/python3.9/site-packages/torch/nn/parallel/data_parallel.py:32: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 1 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the 

Step,Training Loss,Validation Loss,Cer
200,0.816800,0.665364,0.160180
400,0.245100,0.345924,0.016624
600,0.281000,0.282267,0.004023
800,0.319900,0.333758,0.005234
1000,0.275400,0.291335,0.000727
1200,0.184600,0.230665,0.000291
1400,0.060500,0.370400,0.000121


/nfs/home/isaitov/miniconda3/envs/nl39/lib/python3.9/site-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
/nfs/home/isaitov/miniconda3/envs/nl39/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/nfs/home/isaitov/miniconda3/envs/nl39/lib/python3.9/site-packages/torch/nn/parallel/data_parallel.py:32: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 1 which
    has less than 75% of the memory or cores of GPU 0. You can do so

NameError: name 'DataLoader' is not defined

In [8]:
# test eval large Ru
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader
test_dataloader = DataLoader(test_dataset, batch_size=8)
for batch in tqdm(test_dataloader):
    # predict using generate
    pixel_values = batch["pixel_values"].to('cuda' if torch.cuda.is_available() else 'cpu')
    outputs = trainer.model.generate(pixel_values)

    # decode
    pred_str = processor.batch_decode(outputs, skip_special_tokens=True)
    labels = batch["labels"]
    labels[labels == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(labels, skip_special_tokens=True)

    # add batch to metric
    cer_metric.add_batch(predictions=pred_str, references=label_str)

final_score = cer_metric.compute()
print(f"Character error rate on test {key} set:", final_score)

  0%|                                                                                           | 0/356 [00:00<?, ?it/s]/nfs/home/isaitov/miniconda3/envs/nl39/lib/python3.9/site-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (64) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|█████████████████████████████████████████████████████████████████████████████████| 356/356 [07:14<00:00,  1.22s/it]

Character error rate on test Ru set: 0.0013382402141184342
